In [ ]:
import numpy as np
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import nltk
import string
import pickle
from sklearn.svm import SVC
import joblib

from sklearn.feature_extraction.text import CountVectorizer
from sklearn import svm
from sklearn.model_selection import GridSearchCV

nltk.download('punkt')
nltk.download('stopwords')
df = pd.read_csv('spam.csv')
df
encoder = LabelEncoder()
df['Label'] = encoder.fit_transform(df['Label'])
df
df = df.drop_duplicates(keep='first')
df.shape
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
def get_importantFeatures(sent):
    sent = sent.lower()
    
    returnList = []
    sent = nltk.word_tokenize(sent)
    for i in sent:
        if i.isalnum():
            returnList.append(i)
    return returnList
    pass

def removing_stopWords(sent):
    returnList = []
    for i in sent:
        if i not in nltk.corpus.stopwords.words('english') and i not in string.punctuation:
            returnList.append(i)
    return returnList
    pass

def potter_stem(sent):
    returnList = []
    for i in sent:
        returnList.append(ps.stem(i))
    return " ".join(returnList)
    pass
df.loc[:, 'imp_feature'] = df['EmailText'].apply(get_importantFeatures)
df.loc[:, 'imp_feature'] = df['imp_feature'].apply(removing_stopWords)
df.loc[:, 'imp_feature'] = df['imp_feature'].apply(potter_stem)
df
from sklearn.model_selection import train_test_split
X = df['imp_feature']
y = df['Label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
feature = tfidf.fit_transform(X_train)
X_train_tfidf = tfidf.fit_transform(X_train)
# No need to transform X_train, let GridSearchCV handle it internally
tuned_parameters = {'kernel': ['linear', 'rbf'], 'gamma': [1e-3, 1e-4], 'C': [1, 10, 100, 1000]}
model = GridSearchCV(SVC(), tuned_parameters)
model.fit(X_train_tfidf, y_train)
y_predict = tfidf.transform(X_test)
print("Accuracy:",model.score(y_predict,y_test))
best_model = model.best_estimator_
joblib.dump(best_model, "spam_detector.pkl")
import pickle
filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))
from tkinter import *
import tkinter as tk

spam_model = pickle.load(open("finalized_model.sav",'rb'))

def check_spam():
    text = spam_text_Entry.get()
    is_spam = spam_model.predict(tfidf.transform([text]))
    if is_spam == 1:
        print("text is spam")
        my_string_var.set("Result: text is spam")
    else:
        print("text is not spam")
        my_string_var.set("Result: text is not spam")
win = Tk()

win.geometry("400x600")
win.configure(background="cyan")
win.title("Sms Spam Detector")

title = Label(win, text="SMS Spam Detector", bg="gray",width="300",height="2",fg="white",font=("Calibri 20 bold italic underline")).pack()

spam_text = Label(win, text="Enter your Text: ",bg="cyan", font=("Verdana 12")).place(x=12,y=100)
spam_text_Entry = Entry(win, textvariable=spam_text,width=33)
spam_text_Entry.place(x=155, y=105)

my_string_var = StringVar()
my_string_var.set("Result: ")

print_spam = Label(win, textvariable=my_string_var,bg="cyan", font=("Verdana 12")).place(x=12,y=200)

Button = Button(win, text="Submit",width="12",height="1",activebackground="red",bg="Pink",command=check_spam,font=("Verdana 12")).place(x=12,y=150)

win.mainloop()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Accuracy: 0.9737045630317092
